In [ ]:
import geopandas as gpd
import pandas as pd

pd.set_option("display.max_columns", 500)
pd.set_option("display.width", 1000)

# cities = gpd.read_feather(
#     "/Users/test/Documents/code/IL2/factory_loc_service/examples/closest_cities.feather"
# )
cities = gpd.read_parquet(
    "/Users/test/Documents/code/IL2/factory_loc_service/api/app/data/cities.parquet"
)

uinput_industry = "oil_and_gas_ext"
uinput_spec_num = {"Машинист": 200, "Оператор, аппаратчик": 300}
city_names = cities["region_city"].values

In [ ]:
# print(uinput_industry, uinput_spec_num, city_names)

oil_and_gas_ext {'Машинист': 200, 'Оператор, аппаратчик': 300} ['Ростовская область, Аксай' 'Ростовская область, Белая Калитва'
 'Ростовская область, Зверево' 'Ростовская область, Константиновск'
 'Ростовская область, Красный Сулин' 'Ростовская область, Новочеркасск'
 'Ростовская область, Новошахтинск' 'Ростовская область, Семикаракорск'
 'Ростовская область, Шахты']


In [17]:
ontology = pd.read_pickle("new_ontology.pkl")

grouped_grads = pd.read_pickle("grouped_grads.pkl")

cv = pd.read_parquet("../api/app/data/cv.gzip").rename(
    columns={"hh_name": "speciality"}
)
YEAR = 2021
cv = cv[cv["year"] == YEAR]

In [18]:
mask = grouped_grads.reset_index(drop=False).loc[:, "cluster_center"].isin(city_names)
grouped_grads = (
    grouped_grads.reset_index(drop=False)
    .loc[mask, :]
    .set_index(["cluster_center", "region_city", "type", "edu_group_code"])
)

mask = cv["region_city"].isin(city_names)
cv = cv[mask]

In [23]:
# ontology = pd.read_csv(
#     "/Users/test/Documents/code/IL2/industrial-location/api/app/data/ontology.csv",
#     index_col=0,
# )

# ontology.loc[ontology["industry_code"] == "mining_oil_gas", "industry_code"] = (
#     "oil_and_gas_ext"
# )
# ontology.loc[ontology["industry_code"] == "mmining_coal_ores", "industry_code"] = (
#     "coal_and_metal_ext"
# )
# ontology.loc[ontology["industry_code"] == "mech_engineering", "industry_code"] = (
#     "machinery"
# )
# ontology.loc[ontology["industry_code"] == "shipbuilding", "industry_code"] = (
#     "shipbuilding"
# )
# ontology.loc[ontology["industry_code"] == "aircraft_engineering", "industry_code"] = (
#     "aircraft_and_space"
# )
# ontology.loc[ontology["industry_code"] == "non_ferrous_metallurgy", "industry_code"] = (
#     "nonferrous_metallurgy"
# )
# ontology.loc[ontology["industry_code"] == "ferrous_metallurgy", "industry_code"] = (
#     "ferrous_metallurgy"
# )
# ontology.loc[ontology["industry_code"] == "chemical", "industry_code"] = "chemicals"
# ontology.loc[ontology["industry_code"] == "pharma", "industry_code"] = "pharmacy"
# ontology.loc[ontology["industry_code"] == "electronics", "industry_code"] = (
#     "electronics"
# )

In [24]:
# ontology["industry_code"].unique()

In [25]:
# ontology.to_pickle("new_ontology.pkl")

In [19]:
cv = cv.merge(ontology[["speciality", "edu_group_code"]], on="speciality")
cv["type"] = "CV"
grouped_cv = (
    cv.groupby(["cluster_center", "region_city", "type", "edu_group_code"])["id_cv"]
    .count()
    .to_frame()
)
cv.head(3)

,id_cv,speciality,year,region_city,cluster_center,edu_group_code,type
0,c489ca40-355c-11eb-ac41-3bfa22f2d66b,Сварщик,2021,"Башкортостан, Туймазы","Башкортостан, Туймазы",15,CV
1,c489ca40-355c-11eb-ac41-3bfa22f2d66b,Сварщик,2021,"Башкортостан, Туймазы","Башкортостан, Туймазы",15,CV
2,c489ca40-355c-11eb-ac41-3bfa22f2d66b,Сварщик,2021,"Башкортостан, Туймазы","Башкортостан, Туймазы",15,CV


In [27]:
# grads.dropna(subset="edu_group", inplace=True)
# grouped_grads = (
#     grads.groupby(["cluster_center", "region_city", "type", "edu_group_code"])[
#         "graduates_per_year_forecast"
#     ]
#     .sum()
#     .to_frame()
# )
# grouped_grads = grouped_grads.join(grouped_cv, how="outer")
# grouped_grads["city_capacity_grads_and_cv_sum"] = grouped_grads[
#     "graduates_per_year_forecast"
# ].fillna(0) + grouped_grads["id_cv"].fillna(0)
# grouped_grads.to_pickle("grouped_grads.pkl")

In [20]:
# --- START ---
"""
Эта штука завязана на входных профессиях и индустрии,
поэтому нужно каждый раз пересчитывать -- мб можно оптимизировать и пересчитывать только часть
но имхо оно и так быстро
"""

uinput_spec_num_2 = dict()
competitor_industries = list()
competitor_fatories = list()


for k in uinput_spec_num.keys():
    uinput_spec_num_2[k] = (
        ontology.loc[ontology["speciality"] == k, "edu_group_id"]
        .drop_duplicates()
        .values.tolist()
    )
    competitor_industries += (
        ontology.loc[ontology["speciality"] == k, "industry_code"]
        .drop_duplicates()
        .tolist()
    )
# --- START ---

competitor_industries = set(competitor_industries)
competitor_industries.remove(uinput_industry)


grad_col = []
fact_col = []

for col in cities.columns:
    if "factor" in col and uinput_industry in col:
        fact_col.append(col)

    if "factor" in col and any(industry in col for industry in competitor_industries):
        competitor_fatories.append(col)

    if "grad" in col and uinput_industry in col:
        grad_col.append(col)

cities["competitors_factories_num"] = cities[competitor_fatories].sum(axis=1)

print(competitor_industries)
print(grad_col, fact_col, competitor_fatories)

{'mining_coal_ores', 'ferrous_metallurgy', 'pharmacy', 'machinery', 'shipbuilding', 'nonferrous_metallurgy', 'chemicals', 'electronics'}
['graduates_oil_and_gas_ext'] ['factories_oil_and_gas_ext'] ['factories_machinery', 'factories_shipbuilding', 'factories_pharmacy', 'factories_chemicals', 'factories_nonferrous_metallurgy', 'factories_ferrous_metallurgy', 'factories_electronics']


In [ ]:
cv.drop_duplicates(subset=["speciality", "edu_group_code"])

,id_cv,speciality,year,region_city,cluster_center,edu_group_code,type
0,c489ca40-355c-11eb-ac41-3bfa22f2d66b,Сварщик,2021,"Башкортостан, Туймазы","Башкортостан, Туймазы",15,CV
3,c489ca40-355c-11eb-ac41-3bfa22f2d66b,Сварщик,2021,"Башкортостан, Туймазы","Башкортостан, Туймазы",22,CV
8,544e2560-bd46-11eb-b5dd-69632329477c,Монтажник,2021,"Ямало-Ненецкий автономный округ, Надым","Ямало-Ненецкий автономный округ, Надым",21,CV
10,544e2560-bd46-11eb-b5dd-69632329477c,Монтажник,2021,"Ямало-Ненецкий автономный округ, Надым","Ямало-Ненецкий автономный округ, Надым",15,CV
11,544e2560-bd46-11eb-b5dd-69632329477c,Монтажник,2021,"Ямало-Ненецкий автономный округ, Надым","Ямало-Ненецкий автономный округ, Надым",26,CV
...,...,...,...,...,...,...,...
5109,b6b01220-92aa-11eb-b768-e736a3d3ed84,Наладчик,2021,"Самарская область, Тольятти","Самарская область, Тольятти",15,CV
32456,a3ab4880-3fd8-11ec-b1e6-839f0d9a4379,Лаборант,2021,"Ленинградская область, Мурино","Ленинградская область, Мурино",18,CV
32457,a3ab4880-3fd8-11ec-b1e6-839f0d9a4379,Лаборант,2021,"Ленинградская область, Мурино","Ленинградская область, Мурино",21,CV
32459,a3ab4880-3fd8-11ec-b1e6-839f0d9a4379,Лаборант,2021,"Ленинградская область, Мурино","Ленинградская область, Мурино",19,CV


In [14]:
grouped_grads = (
    grouped_grads.reset_index(drop=False)
    .groupby(["cluster_center", "type"])[["city_capacity_grads_and_cv_sum", "id_cv"]]
    .sum()
    .join(
        cities[
            ["region_city", "population", "num_in_migration"]
            + grad_col
            + fact_col
            + ["factories_total"]
            + ["competitors_factories_num"]
        ]
        .rename(columns={"region_city": "cluster_center"})
        .set_index("cluster_center"),
        how="left",
    )
)

grouped_grads[
    [
        "num_in_migration",
        "graduates_oil_and_gas_ext",
        "factories_oil_and_gas_ext",
        "factories_total",
        "city_capacity_grads_and_cv_sum",
        "population",
        "competitors_factories_num",
    ]
] = (
    grouped_grads[
        [
            "num_in_migration",
            "graduates_oil_and_gas_ext",
            "factories_oil_and_gas_ext",
            "factories_total",
            "city_capacity_grads_and_cv_sum",
            "population",
            "competitors_factories_num",
        ]
    ]
    .round(0)
    .fillna(0)
    .astype(int)
)

grouped_grads["working_population"] = (
    (grouped_grads["population"] * 0.65).round(0).astype(int)
)

In [30]:
"""
(!) Сейчас есть всё кроме непосредственной оценки (!)
А ну и задаваемое кол-во специалистов пока не учитывается
Но это следствие из отсутствия реализации расчета оценки

Эту таблицу практически в таком виде и планирую отдавать

Из того что обсуждали с СА:
условно выпускников взвесить по кол-ву предприятий тк их число ту мач большое
"""

'\n(!) Сейчас есть всё кроме непосредственной оценки (!)\nА ну и задаваемое кол-во специалистов пока не учитывается\nНо это следствие из отсутствия реализации расчета оценки\n\nЭту таблицу практически в таком виде и планирую отдавать\n\nИз того что обсуждали с СА:\nусловно выпускников взвесить по кол-ву предприятий тк их число ту мач большое\n'

In [15]:
grouped_grads

city_capacity_grads_and_cv_sum   id_cv  population  num_in_migration  graduates_oil_and_gas_ext  factories_oil_and_gas_ext  factories_total  competitors_factories_num  working_population
cluster_center                 type                                                                                                                                                                                            
Адыгея, Адыгейск               CV                               118   118.0       13191                 4                          0                          0                0                          0                8574
Адыгея, Майкоп                 CV                               260   260.0      139687                20                        554                          0                6                          5               90797
                               ВПО                             1993     0.0      139687                20                        554                          0                6                          5               90797
                               СПО                              660     0.0      139687                20                        554                          0                6                          5               90797
Алтай, Горно-Алтайск           CV                               508   508.0       64957                 7                        468                          0                1                          1               42222
...                                                             ...     ...         ...               ...                        ...                        ...              ...                        ...                 ...
Ярославская область, Углич     CV                               226   226.0       32228                 4                          0                          0                5                          5               20948
                               СПО                               40     0.0       32228                 4                          0                          0                5                          5               20948
Ярославская область, Ярославль CV                              2134  2134.0      570824               443                        559                          0               65                         65              371036
                               ВПО                             5723     0.0      570824               443                        559                          0               65                         65              371036
                               СПО                             1307     0.0      570824               443                        559                          0               65                         65              371036

[2111 rows x 9 columns]

In [32]:
""" ------------------------- FINISH HERE -------------------------------- """

' ------------------------- FINISH HERE -------------------------------- '

In [1]:
import pandas as pd

In [4]:
gg = pd.read_pickle(
    "/Users/test/Documents/code/IL2/factory_loc_service/api/app/data/grouped_grads.pkl"
).reset_index()

In [5]:
gg

,cluster_center,region_city,type,edu_group_code,speciality,graduates_per_year_forecast,cv_count,city_capacity_grads_and_cv_sum
0,"Адыгея, Адыгейск","Адыгея, Адыгейск",CV,5,Геолог,NaN,2.0,2.0
1,"Адыгея, Адыгейск","Адыгея, Адыгейск",CV,11,"Оператор, аппаратчик",NaN,2.0,2.0
2,"Адыгея, Адыгейск","Адыгея, Адыгейск",CV,11,Электромонтажник,NaN,1.0,1.0
3,"Адыгея, Адыгейск","Адыгея, Адыгейск",CV,15,Инженер-проектировщик,NaN,1.0,1.0
4,"Адыгея, Адыгейск","Адыгея, Адыгейск",CV,15,"Оператор, аппаратчик",NaN,4.0,4.0
...,...,...,...,...,...,...,...,...
110616,"Ярославская область, Ярославль","Ярославская область, Ярославль",СПО,18,NaN,59.0,NaN,59.0
110617,"Ярославская область, Ярославль","Ярославская область, Ярославль",СПО,19,NaN,309.0,NaN,309.0
110618,"Ярославская область, Ярославль","Ярославская область, Ярославль",СПО,21,NaN,120.0,NaN,120.0
110619,"Ярославская область, Ярославль","Ярославская область, Ярославль",СПО,22,NaN,41.0,NaN,41.0


In [9]:
df_pivot = gg.pivot_table(
    index="cluster_center",
    columns="type",
    values=[
        "graduates_per_year_forecast",
        "cv_count",
        "city_capacity_grads_and_cv_sum",
    ],
    aggfunc="sum",
).reset_index()

# The result is a DataFrame that has:
# cluster_center    CV       ВПО       СПО
# Адыгея, Адыгейск  118.0    NaN       NaN
# Адыгея, Майкоп     260.0    1993.0    660.0
# Алтай, Горно-Алтайск 508.0 NaN       NaN
# ... and so on

# If you want to rename `cluster_center` to `region_name`:
df_pivot = df_pivot.rename(columns={"cluster_center": "region_city"})

# To fill any missing values with 0 (in case some types are not present for a region):
df_pivot = df_pivot.fillna(0)
df_pivot

region_city city_capacity_grads_and_cv_sum          \
type                                                             CV     ВПО   
0                   Адыгея, Адыгейск                          118.0     0.0   
1                     Адыгея, Майкоп                          260.0  1993.0   
2               Алтай, Горно-Алтайск                          508.0   979.0   
3             Алтайский край, Алейск                         1807.0     0.0   
4            Алтайский край, Барнаул                         3135.0  8265.0   
...                              ...                            ...     ...   
1100     Ярославская область, Ростов                          197.0     0.0   
1101    Ярославская область, Рыбинск                         1047.0  1662.0   
1102     Ярославская область, Тутаев                          277.0     0.0   
1103      Ярославская область, Углич                          226.0     0.0   
1104  Ярославская область, Ярославль                         2134.0  5723.0   

             cv_count           graduates_per_year_forecast                  
type     СПО       CV  ВПО  СПО                          CV     ВПО     СПО  
0        0.0    118.0  0.0  0.0                         0.0     0.0     0.0  
1      660.0    260.0  0.0  0.0                         0.0  1993.0   660.0  
2      318.0    508.0  0.0  0.0                         0.0   979.0   318.0  
3       96.0   1807.0  0.0  0.0                         0.0     0.0    96.0  
4     1636.0   3135.0  0.0  0.0                         0.0  8265.0  1636.0  
...      ...      ...  ...  ...                         ...     ...     ...  
1100    69.0    197.0  0.0  0.0                         0.0     0.0    69.0  
1101   526.0   1047.0  0.0  0.0                         0.0  1662.0   526.0  
1102    15.0    277.0  0.0  0.0                         0.0     0.0    15.0  
1103    40.0    226.0  0.0  0.0                         0.0     0.0    40.0  
1104  1307.0   2134.0  0.0  0.0                         0.0  5723.0  1307.0  

[1105 rows x 10 columns]